In [ ]:
import numpy as np
from numpy.core.fromnumeric import transpose
import pandas as pd

In [ ]:
df = pd.read_csv('COMPAS_preprocessed.csv')
df.head(5)

,Unnamed: 0,age,c_charge_degree,race,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out,days_in_jail
0,1,34,0,1,-1,1,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53,10
1,2,24,0,1,-1,1,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04,1
2,6,41,0,0,0,1,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30,6
3,8,39,1,0,-1,0,0,-1.0,1,0,0,2014-03-15 05:35:34,2014-03-18 04:28:46,2
4,10,27,0,0,-1,1,0,-1.0,4,0,0,2013-11-25 06:31:06,2013-11-26 08:26:57,1


In [ ]:
cols =  ['age', 'c_charge_degree', 'race', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid',
       'two_year_recid', 'days_in_jail']
df = df[cols]
df.head(5)

,age,c_charge_degree,race,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,days_in_jail
0,34,0,1,-1,1,0,-1.0,3,1,1,10
1,24,0,1,-1,1,4,-1.0,4,1,1,1
2,41,0,0,0,1,14,-1.0,6,1,1,6
3,39,1,0,-1,0,0,-1.0,1,0,0,2
4,27,0,0,-1,1,0,-1.0,4,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split
col_X = ['age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 
        'days_in_jail']
X = df[col_X]
Z = df['race']
y = df['two_year_recid']
X_train, X_test, Z_train, Z_test, y_train, y_test = train_test_split( X, Z, y, test_size=0.33, random_state=42)

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
#https://scipy-lectures.org/advanced/mathematical_optimization/auto_examples/plot_non_bounds_constraints.html
import scipy
from scipy.optimize import minimize
from numpy import linalg as LA

k = X.shape[1]
N = X.shape[0]

def upper_theta_constraint(params, X, Z, c, k):
  theta = params[:k]
  return ((-1/len(Z)) * np.matmul(np.matmul(transpose(Z - Z.mean()), X), theta)) + c

def lower_theta_constraint(params, X, Z, c, k):
  theta = params[:k]
  return ((np.matmul(np.matmul(transpose(Z - Z.mean()), X), theta))/len(Z)) + c  

def svm_loss(params,C,k):
  theta = params[:k]
  phi = params[k:]
  return LA.norm(theta,2) + sum(phi) * C

def phi_constraint(params,k):
  #theta = params[:k]
  phi = params[k:]

  return phi 

def phi_constraint2(params, k, y , X):
  theta = params[:k]
  phi = params[k:]
  return np.dot( transpose(theta), np.matmul(y,X)) - 1 + sum(phi)

In [ ]:
theta = np.array(np.random.uniform(size=k)).reshape(-1, 1)
phi = np.array(np.random.uniform(size=N))
params = np.append(theta.flatten(), phi.flatten())
scipy.optimize.minimize(svm_loss, x0=params, args=(0.8, k), 
                        method='SLSQP', 
                        constraints=({'type': 'ineq', 'fun': upper_theta_constraint, 'args': (X_train, Z_train , 0.8, k)},
                                     {'type': 'ineq', 'fun': lower_theta_constraint, 'args': (X_train, Z_train , 0.8, k)},
                                      {'type': 'ineq', 'fun': phi_constraint, 'args': [k]},
                                     {'type': 'ineq', 'fun': phi_constraint2, 'args': [k, y_train, X_train]}),
                  
                        options={"maxiter": 10})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, n

     fun: 0.00013724511568326291
     jac: array([ 0.15974887, -0.35147453, -0.48540974, ...,  0.8       ,
        0.8       ,  0.8       ])
 message: 'Iteration limit reached'
    nfev: 52894
     nit: 10
    njev: 10
  status: 9
 success: False
       x: array([ 2.20358939e-05, -4.85051980e-05, -6.69855863e-05, ...,
       -1.60466310e-10, -5.11471327e-10, -1.04626359e-10])